In [3]:
import functools
import itertools
import pathlib
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
from torchsummary import summary
from matplotlib import rc

%matplotlib inline

%load_ext autoreload
%autoreload 2

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
n_cmds = 10
n_files = 3
local_ranks = [0, 1]
local_ranks_iter = itertools.cycle(local_ranks)
cmds = []

for i in range(1, n_cmds + 1):
    local_rank = next(local_ranks_iter)
    cmd = f'cmd --id {i} --local_rank {local_rank} MODEL.DEVICE \"CUDA:{local_rank}\"'
    cmds.append(cmd)

def get_cmd_cuda_device(cmd):
    cuda_str = "CUDA:"
    pos = cmd.find(cuda_str)
    
    if pos < 0:
        device_id = 0
    else:
        id_pos = pos + len(cuda_str)
        device_id = int(cmd[id_pos:id_pos + 1])
    
    return device_id

cmds_sorted = sorted(cmds, key=get_cmd_cuda_device)
for cuda_device, cmd_cuda_group in itertools.groupby(cmds_sorted, key=get_cmd_cuda_device):
    print(f"Device: {cuda_device}")
    for cmd in cmd_cuda_group:
        print(f"\t{cmd}")
    

Device: 0
	cmd --id 1 --local_rank 0 MODEL.DEVICE "CUDA:0"
	cmd --id 3 --local_rank 0 MODEL.DEVICE "CUDA:0"
	cmd --id 5 --local_rank 0 MODEL.DEVICE "CUDA:0"
	cmd --id 7 --local_rank 0 MODEL.DEVICE "CUDA:0"
	cmd --id 9 --local_rank 0 MODEL.DEVICE "CUDA:0"
Device: 1
	cmd --id 2 --local_rank 1 MODEL.DEVICE "CUDA:1"
	cmd --id 4 --local_rank 1 MODEL.DEVICE "CUDA:1"
	cmd --id 6 --local_rank 1 MODEL.DEVICE "CUDA:1"
	cmd --id 8 --local_rank 1 MODEL.DEVICE "CUDA:1"
	cmd --id 10 --local_rank 1 MODEL.DEVICE "CUDA:1"


In [53]:
from math import ceil, floor

n_files = 6
n_files_portion_pool = float(n_files)
group_sizes = [12, 4]
for group_size in group_sizes:
    n_curr_files_portion = (group_size / sum(group_sizes)) * n_files
    n_curr_files = int(round(n_curr_files_portion))
    print(n_curr_files, n_curr_files_portion)

ImportError: cannot import name 'divmod'

In [51]:
np.array_split(cmds, 3)

[array(['cmd --id 1 --local_rank 0 MODEL.DEVICE "CUDA:0"',
        'cmd --id 2 --local_rank 1 MODEL.DEVICE "CUDA:1"',
        'cmd --id 3 --local_rank 0 MODEL.DEVICE "CUDA:0"',
        'cmd --id 4 --local_rank 1 MODEL.DEVICE "CUDA:1"'], dtype='<U48'),
 array(['cmd --id 5 --local_rank 0 MODEL.DEVICE "CUDA:0"',
        'cmd --id 6 --local_rank 1 MODEL.DEVICE "CUDA:1"',
        'cmd --id 7 --local_rank 0 MODEL.DEVICE "CUDA:0"'], dtype='<U48'),
 array(['cmd --id 8 --local_rank 1 MODEL.DEVICE "CUDA:1"',
        'cmd --id 9 --local_rank 0 MODEL.DEVICE "CUDA:0"',
        'cmd --id 10 --local_rank 1 MODEL.DEVICE "CUDA:1"'], dtype='<U48')]